In [179]:
from pymongo import MongoClient
from pprint import pprint
from pprint import pprint
import math
import json
import pymongo
from tqdm import tqdm
from pymongo import GEOSPHERE

In [108]:
client = MongoClient()
db = client['Restaurant_Recommend']
restaurants = db['restaurant']
reviews = db['review']

In [79]:
def calculate_adjusted_score(review):
    rating = review['stars']
    upvotes = review['useful']
    if rating < 3:
        return rating - math.log(upvotes)
    elif rating > 3:
        return rating + math.log(upvotes)

    else:
        return rating


In [161]:
calculate_usefulness_score(j)

8.912023005428146

In [160]:
j = {
 'business_id': 'Pns2l4eNsfO8kk83dixA6A',
 'cool': 1,
 'date': '2012-05-02 18:07:38',
 'funny': 2,
 'review_id': '9vwYDBVI3ymdqcyJ5WW2Tg',
 'stars': 5,
 'text': "I've had acupuncture treatments with Abby over the past few years "
         "and I really can't say enough about how great my experiences have "
         'been.\n'
         '\n'
         'She has successfully treated me for everything from stress to colds '
         'to menstrual cramps.  I was amazed that her medical qi gong '
         "treatments helped alleviate my chronic back pain.  I didn't think "
         'anything could help with that!  \n'
         '\n'
         'She is always very clear and upfront about treatments and '
         'expectations.   And I love that she always has great advice on '
         'herbs, supplements and natural remedies.\n'
         '\n'
         "She just relocated to Los Gatos and I'm definitely going to miss "
         'having her in San Francisco!',
 'useful': 50,
 'user_id': 'e0imecnX_9MtLnS2rUZM-A'}

In [170]:
calculate_usefulness_score(j)

8.912023005428146

In [82]:
# Update each review with an adjusted score

pipeline = [
    {
        "$addFields": {
            "adjusted_score": {
                "$switch": {
                    "branches": [
                        {
                            "case": { "$lt": [ "$stars", 3 ] },
                            "then": { "$subtract": [ "$stars", { "$cond": [{ "$gt": ["$useful", 0]}, { "$ln": "$useful"}, 0] }] }
                        },
                        {
                            "case": { "$gt": [ "$stars", 3 ] },
                            "then": { "$add": [ "$stars", { "$cond": [{ "$gt": ["$useful", 0]}, { "$ln": "$useful"}, 0] }] }
                        }
                    ],
                    "default": "$stars"
                }
            }
        }
    }
]

reviews.update_many({}, pipeline)

KeyboardInterrupt: 

In [146]:
pipeline = [
    {"$group": {"_id": "$business_id", "average_adj_score": {"$avg": "$adjusted_score"}}}
]
result = list(reviews.aggregate(pipeline))

# create list of bulk write operations
operations = []
for pair in result:
    operations.append(
        pymongo.UpdateOne({'business_id': pair['_id']}, {'$set': {'average_adj_score': pair['average_adj_score']}})
    )

# perform bulk write operation
restaurants.bulk_write(operations)

In [166]:
avg_adj_score = 1
num_reviews = 10
(avg_adj_score * num_reviews) / (num_reviews + 5)

0.6666666666666666

In [147]:
pipeline = [
    {"$addFields": {
        "credibility_score": {
            "$divide": [
                {"$multiply": ["$average_adj_score", "$review_count"]},
                {"$add": ["$review_count", 5]}
                ]
            }
        }
    }
]

restaurants.update_many({}, pipeline)

In [177]:
pipeline = [
    {'$addFields': {
        'coord': ['$latitude', '$longitude']
    }}
]

restaurants.update_many({}, pipeline)

In [181]:
restaurants.create_index([('restaurant.coord', GEOSPHERE)])

'restaurant.coord_2dsphere'

In [183]:

def find_closest(coords):
    pipeline = [
        {'$geoNear' : {
            'distanceField': 'dist.calculated',
            'maxDistance': 100000000,
            'near': {'type': "Point", 'coordinates': coords},
            'spherical': True

            }
        },
        {'$sort': {'dist.calculated': 1}
         },
        {
        '$project': {
            'name': 1,
            'address': 1

        }
        },
        {
        '$limit': 10
        }
    ]
    return list(restaurants.aggregate(pipeline))

In [178]:
restaurants.find_one()

{'_id': ObjectId('6428a1c1c0e805b4f6db16a3'),
 'business_id': 'Pns2l4eNsfO8kk83dixA6A',
 'name': 'Abby Rappoport, LAC, CMQ',
 'address': '1616 Chapala St, Ste 2',
 'city': 'Santa Barbara',
 'state': 'CA',
 'postal_code': '93101',
 'latitude': 34.4266787,
 'longitude': -119.7111968,
 'stars': 5,
 'review_count': 7,
 'is_open': 0,
 'attributes': {'ByAppointmentOnly': 'True'},
 'categories': 'Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
 'hours': None,
 'average_adj_score': 5.212129521398286,
 'credibility_score': 3.0404088874823336,
 'coord': [34.4266787, -119.7111968]}